### Dataset link: http://medicalsegmentation.com/covid19/

Segmentation dataset nr. 2 (13th April)

* ground-glass (mask value =1)
* consolidation (=2) 
* pleural effusion (=3)

In [ ]:
# Import libraries

import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors
%matplotlib inline
import nibabel as nib
import numpy as np
from skimage.transform import resize
import scipy.misc
from skimage.transform import rotate
from skimage.io import imread, imshow, imread_collection, concatenate_images

ROOT_DIR = os.path.abspath("../")
DATASET_NAME = "COVID19v2"
DATASET_PATH = os.path.join(ROOT_DIR, "datasets", DATASET_NAME)
print(os.listdir(DATASET_PATH))

In [ ]:
def hu_window(image):
    "Normalize"
    
    # For lung
    #############
    MIN_HU = np.min(image) #-512
    MAX_HU = np.max(image) #512
    #############
    
    image[image < MIN_HU] = MIN_HU
    image[image > MAX_HU] = MAX_HU
    image = (image - MIN_HU) / (MAX_HU - MIN_HU)
    image = image.astype("float32")
    return image


def rotate_img(arr):
    arr = cv2.rotate(arr, cv2.ROTATE_90_CLOCKWISE)
    return arr
    
    
def hu_to_grayscale(volume):
    volume = np.clip(volume, -512, 512)
    mxval  = np.max(volume)
    mnval  = np.min(volume)
    im_volume = (volume - mnval)/max(mxval - mnval, 1e-3)
    im_volume = im_volume
    return im_volume *255


# Helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    
    norm=plt.Normalize(0,3) # 4 classes including BG
    map_name = matplotlib.colors.LinearSegmentedColormap.from_list("", ["black", "red","green","blue"])

    
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        #plt.imshow(image, cmap=map_name, norm=norm)
        plt.imshow(image)
    plt.show()

In [ ]:
vol_path = DATASET_PATH + "/rp_im" # tr_mask.nii.gz
mask_path = DATASET_PATH + "/rp_msk"

vols = [os.path.join(vol_path, x) for x in os.listdir(vol_path)]
masks = [os.path.join(mask_path, x) for x in os.listdir(mask_path)]

print(vol_path, mask_path)
print(len(vols), len(masks))

In [ ]:
# Read CT
vol = nib.load(vols[0])
masks = nib.load(masks[0])

In [ ]:
vol = np.asanyarray(vol.dataobj)
masks = np.asanyarray(masks.dataobj)
vol.shape, masks.shape

In [ ]:
np.unique(masks[:,:,0])

In [ ]:
#vol = rotate(vol, -90, resize=False)  
#masks = rotate(masks, -90, resize=False)  

In [ ]:
#np.unique(masks[:,:,0])

In [ ]:
# Normalize
vol = hu_window(vol)
vol.shape

In [ ]:
ct = 60
img = rotate_img(vol[:,:,ct])
plt.imshow(img, cmap='gray')

In [ ]:
np.min(vol[:,:,ct])

In [ ]:
img.shape

In [ ]:
x = cv2.resize(img, (512, 512))
x.shape

In [ ]:
norm=plt.Normalize(0,3) # 4 classes including BG
map_name = matplotlib.colors.LinearSegmentedColormap.from_list("", ["black", "red","green","blue"])
msk = rotate_img(masks[:,:,ct])
plt.imshow(msk, cmap=map_name, norm=norm)

In [ ]:
np.unique(msk)

In [ ]:
CLASSES = ['groundglass', 'consolidation', 'pleuraleffusion']
class_values = [CLASSES.index(cls) for cls in CLASSES]
class_values

In [ ]:
# Extract certain classes from mask
masks = [(masks == v) for v in class_values]
masks = np.stack(masks, axis=-1).astype('float')
# Add background if mask is not binary
if masks.shape[-1] != 1:
    background = 1 - masks.sum(axis=-1, keepdims=True)
    masks = np.concatenate((masks, background), axis=-1)
    
masks.shape

In [ ]:
ct = 8

visualize(
    image=vol[:,:,ct], 
    GroundGlass_mask = masks[:,:,ct,1],
    Consolidation_mask = masks[:,:,ct,2],
    PleuralEffusion_mask = masks[:,:,ct,3],
    background_mask = masks[:,:,ct,0],
)

In [ ]:
images = []
maps = []

for idx in range(100):
    
    # Resize image
    img = vol[:,:,idx] 
    img = rotate_img(img)
    img = np.expand_dims(img, axis=-1)
    images.append(img)
    
    
    # Resize masks
    msk = masks[:,:,idx]
    msk = rotate_img(msk)
    maps.append(msk)
    
images = np.array(images)
maps = np.array(maps)

print(images.shape, maps.shape)    

In [ ]:
maps[0,:,:,2].shape

In [ ]:
a = np.argmax(maps[0].squeeze(), axis=-1)
a.shape

In [ ]:
plt.imshow(a)

In [ ]:
np.unique(a)

In [ ]:
for i in range(100):

    ct = i #55

    visualize(
        image=images[ct].squeeze(), 
        GroundGlass_mask = maps[ct,:,:,0],
        Consolidation_mask = maps[ct,:,:,1],
        PleuralEffusion_mask = maps[ct,:,:,2],
        background_mask = maps[ct,:,:,3],
    )

In [ ]:
images.shape, maps.shape

In [ ]:
np.save("{}/COVID19_images.npy".format(DATASET_PATH), images)
np.save("{}/COVID19_mask.npy".format(DATASET_PATH), maps)